# 필요 모듈 및 라이브러리 설치

In [ ]:
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

!pip install sentencepiece
!pip install nltk

%tensorflow_version 1.x

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import logging
from tensorflow.saved_model import tag_constants
import base64
import json

tf.__version__

Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155332 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 1.2 MB 5.5 MB/s 
TensorFlow 1.x selected.


'1.15.2'

In [13]:
# bert 모델 다운
# !git clone https://github.com/google-research/bert /content/Gdrive/bert_files/bert_model
# !git clone https://github.com/google-research/bert
sys.path.append('bert_model')

In [61]:
# 환경변수에 bert 모델 등록돼 있는지 확인
'bert_model' in sys.path

True

# Google Drive와 GCP 연동

In [ ]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the 

In [ ]:
# Gdrive 폴더 만들기
!mkdir -p Gdrive

# google drvie와 colab 연동시키기
!google-drive-ocamlfuse Gdrive -o nonempty

# 파일 리스트 확인
!ls

adc.json  Gdrive  sample_data


# TPU Address 확인

In [32]:
# 현재 colab 런타임 환경이 TPU로 세팅돼 있는지 확인
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'

# TPU 주소 추출
TPU_ADDRESS = 'grpc://'+os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

# 코랩의 인증 모듈로 할당된 TPU 모델 확인하기
from google.colab import auth
auth.authenticate_user()

with tf.Session(TPU_ADDRESS) as session:
    print('TPU devices:')
    pprint.pprint(session.list_devices())

    # TPU에 credentials 정보 저장하기
    with open('/content/adc.json', 'r') as f :
        auth_info = json.load(f)
    
    # google cloud storage(bucket)에 현재 TPU와 인증 정보 설정하기
    # 이 세팅을 해야 gcp bucket과 연동 가능
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

TPU address is grpc://10.37.12.98:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1393923288021924419),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8776013666702480619),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6578782341962919004),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1067138041544584425),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10419007226085709134),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 2478460990524212225),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 6368481881512452367),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 15861108127096795405),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 209585820727694

# 환경변수 세팅

In [39]:
# 학습할 모델 경로
MODEL_DIR = 'gs://nlp_pretrain_model/bert_model/new_model'

# 저장할 모델 경로(output)
PROD_MODEL_DIR='gs://nlp_pretrain_model/output'

# 단어사전 파일 경로
VOCAB_FILE= 'gs://nlp_pretrain_model/bert_model/vocab.txt'

# BERT 최대 길이
MAX_SEQ_LENGTH= 128

# 학습된 모델 경로
PRE_TRAINED_MODEL_PATH='gs://nlp_pretrain_model/bert_model'

# 구글 드라이브 bert 모델 경로
GDRIVE_BERT_ROOT= 'Gdrive/bert_files'

# 구글드라이브 자연어 데이터 경로
GDRIVE_NLP_DATA_ROOT= 'Gdrive/bert_files'

# BERT 환경설정
BERT_CONFIG='gs://nlp_pretrain_model/bert_model/bert_config.json'

# 배치 크기
BATCH_SIZE=64

# 학습 데이터 가공 및 생성

In [ ]:
import pandas as pd
import gspread

gc = gspread.authorize(GoogleCredentials.get_application_default())

FILE_PATH = '/content/Gdrive/ONe/db/Q&A_개발활용가능_2020.10_sample.xlsx'

df = pd.read_excel(FILE_PATH)


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# 컬럼명 변경 (종류)
re_columns = ['번호', '종류', '대분류', '중분류', '소분류', 'Question',
       'Answer', 'Hashtag 2', 'Unnamed: 8', '건강/의료', '환경/생활', '교육/심리',
       '단순문의', '반려상품', '법률', '입양/장례', 'Unnamed: 16', '총합']
df.columns = re_columns

df.head()

- 카테고리 별로 분류 파싱하여 카테고리 트리 만들기

In [ ]:
category_dict = {}
category_list = []

category_list.append(0)
idx=1

for i, row in df.iterrows() :

    if row['종류'] != '개' and row['종류'] != '고양이':
        continue
    if row['종류'] not in category_dict :
        category_dict[row['종류']] = {}
    if row['대분류'] not in category_dict[row['종류']]:
        category_dict[row['종류']][row['대분류']] = {}
    if row['중분류'] not in category_dict[row['종류']][row['대분류']]:
        category_dict[row['종류']][row['대분류']][row['중분류']] = {}
    if row['소분류'] not in category_dict[row['종류']][row['대분류']][row['중분류']]:
        category_dict[row['종류']][row['대분류']][row['중분류']][row['소분류']] = idx

        category_list.append(str(row['종류']) + ':' + str(row['대분류']) + ':' + str(row['중분류']) + ':' + str(row['소분류']))
        idx += 1

print(category_dict)
print(category_list)
print(len(category_list))

{'개': {'건강/의료': {'기타': {nan: 1}, '치료': {'간단건강체크': 2, '구조/구호': 3, '수술': 4, '예방/약': 5, '예방접종': 6, '응급처치': 7, '중성화': 8, '질병/증상': 9, '질병/치료': 10}}, '교육/심리': {'감정/심리': {nan: 11}, '놀이/산책': {nan: 12}, '행동': {nan: 13}, '훈련/행동교정': {nan: 14}}, '단순문의': {'기타': {nan: 15}, '보호자 고민': {nan: 16}, '품종문의': {nan: 17}}, '반려상품': {'기본용품': {'목욕/미용용품': 18}, '식품': {'건강관리': 19, '사료/간식': 20}}, '법률': {'사람/동물': {nan: 21}, '의료사고': {nan: 22}}, '입양/장례': {'기타': {nan: 23}, '입양/분양': {'기타': 24, '입양 받을 때': 25, '입양 보낼 때': 26, '입양받을때': 27}, '장례': {'장례': 28}}, '환경/생활': {'교배/출산': {'교배': 29, '임신/출산': 30}, '급여/간식': {'급여방법': 31, '기타': 32}, '기타': {nan: 33}, '놀이/산책': {nan: 34}, '양육': {'기타': 35, '목욕/미용': 36, '양육': 37, nan: 38}, '환경/생활': {'기타': 39}}}, '고양이': {'건강/의료': {'기타': {nan: 40}, '치료': {'간단건강체크': 41, '구조/구호': 42, '수술': 43, '예방/약': 44, '예방접종': 45, '응급': 46, '응급처치': 47, '중성화': 48, '질병/증상': 49, '질병/치료': 50}}, '교육/심리': {'감정/심리': {nan: 51}, '기타': {nan: 52}, '놀이/산책': {nan: 53}, '행동': {nan: 54}, '훈련/행동교정': {nan: 55}}, '단순문의': {'기타': {

In [8]:
# NUM_LABELS = len(category_list)
NUM_LABELS = 76

### 데이터 전처리

In [ ]:
import re

PATTERN = '[^\w\s]'
PATTERN_MULTI_SPACE = ' +'
PATTERN_LINE_BREAKER = '\n|\r'
REPL = " "

def isNaN(string):
    return string != string

train_raw = {
    'label': [],
    'alpha': [],
    'text': []
}

continued_value = {
    '종류' : [],
    '대분류' : [],
    '중분류' : [],
    '소분류' : [],
    'Question' : []
}

# Question
for i, row in df.iterrows():
    if isNaN(row['Question']) : row['Question'] = REPL
    if row['종류'] != '개' and row['종류'] != '고양이' and row['Question'] != '' and row['Question'] != 'None':
        continue
    else:
        try:
            qu = re.sub(pattern=PATTERN, repl=REPL, string=str(row['Question']))
            qu = re.sub(pattern=PATTERN_LINE_BREAKER, repl=REPL, string=qu)
            qu = re.sub(pattern=PATTERN_MULTI_SPACE, repl=REPL, string=qu)

            if(qu != REPL) : qu = qu.strip()
            train_raw['label'].append(category_dict[row['종류']][row['대분류']][row['중분류']][row['소분류']])
            train_raw['alpha'].append('a')
            train_raw['text'].append(qu)
        
        except:
            continued_value['종류'].append(row['종류'])
            continued_value['대분류'].append(row['대분류'])
            continued_value['중분류'].append(row['중분류'])
            continued_value['소분류'].append(row['소분류'])
            continued_value['Question'].append(row['Question'])

# Answer
for i, row in df.iterrows():
    if isNaN(row['Answer']): 
        row['Answer'] = REPL
    if row['종류'] != '개' and row['종류'] != '고양이' and row['Answer'] != '' and row['Answer'] != 'None':
        continue
    else :
        try:
            qu = re.sub(pattern=PATTERN, repl=REPL, string=str(row['Answer']))
            qu = re.sub(pattern=PATTERN_LINE_BREAKER, repl=REPL, string=qu)
            qu = re.sub(pattern=PATTERN_MULTI_SPACE, repl=REPL, string=qu)
            if(qu != REPL) : qu = qu.strip()
            qu = qu[:256] if len(qu) > 256 else qu 
            train_raw['label'].append(category_dict[row['종류']][row['대분류']][row['중분류']][row['소분류']])
            train_raw['alpha'].append('a')
            train_raw['text'].append(qu)
        except:
            continued_value['종류'].append(row['종류'])
            continued_value['대분류'].append(row['대분류'])
            continued_value['중분류'].append(row['중분류'])
            continued_value['소분류'].append(row['소분류'])
            continued_value['Question'].append(row['Question'])

train =pd.DataFrame(train_raw)

train = train.reset_index() # ==> index 재설정
train = train.rename(columns={'index':'id'}) # index라는 column을 id로 변동

# 에러 난 값들
error_values = pd.DataFrame(continued_value)

In [ ]:
train.head(2)

,id,label,alpha,text
0,0,1,a,감기 걸린 강아지도 산책을 시켜야 하나요
1,1,1,a,암컷 강아지의 생리와 혈뇨는 어떻게 구분하나요


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2136 entries, 0 to 2135
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2136 non-null   int64 
 1   label   2136 non-null   int64 
 2   alpha   2136 non-null   object
 3   text    2136 non-null   object
dtypes: int64(2), object(2)
memory usage: 66.9+ KB


### 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np

df_bert_train, df_bert_dev = train_test_split(train,test_size=0.1)

test = df_bert_dev[['id', 'text']]

test.head()
test.info()

df_bert_test =test
df_bert_test

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 126 to 969
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      214 non-null    int64 
 1   text    214 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.0+ KB


,id,text
126,126,털을 밀고 기르는 중인데 일부분이 빳빳하고 누렇게 변색되었는데 왜 그럴까요
1755,1755,고양이 턱드름을 손으로 짜거나 칫솔로 문지르게 되면 2차 감염으로 염증을 유발 할 ...
594,594,강아지를 비행기에 태우려면 동물등록을 꼭 해야 하나요
43,43,새끼강아지 인데 예방접종 1차까지 완료했는데 산책 언제 해도 될까요
1763,1763,자주 다니는 길 이라면 퇴근할 때 마다 잠깐씩 들러서 찾아보세요 고양이가 다쳤다면 ...
...,...,...
1232,1232,장이 아파서 그럴 수 있어요 구토 원인으로 급성위염 기생충감염 췌장염 장폐색 장중첩...
629,629,새끼고양이 심장에서 그르렁 진동소리가 느껴지는데 정상인가요
1858,1858,고양이가 골골소리를 낼 경우는기분이 좋거나 편안할 때 내는 소리예요 일부에서 불안정...
708,708,고양이 중성화수술 몇개월 때 하면 될까요


# Generate train, dev & test file
- test file header=True (컬럼명을 살린다는 의미)
- other files header=false (컬럼명을 없앤다는 의미)

In [ ]:
df_bert_train.to_csv(GDRIVE_BERT_ROOT + '/data/train.tsv', sep='\t', index=False, header=False, encoding='UTF-8')
df_bert_dev.to_csv(GDRIVE_BERT_ROOT + '/data/dev.tsv', sep='\t', index=False, header=False, encoding='UTF-8')
df_bert_test.to_csv(GDRIVE_BERT_ROOT + '/data/test.tsv', sep='\t', index=False, header=True, encoding='UTF-8')
error_values.to_csv(GDRIVE_BERT_ROOT + '/data/error_values.tsv', sep='\t', index=False, header=True, encoding='UTF-8')

with open('{}/category.json'.format(GDRIVE_BERT_ROOT + '/data'), 'w') as fo:
    json.dump(category_list, fo, indent=2)

# Classify 학습 트레이닝
- Fine-tuning for Classifier
- Added checkpoint

In [15]:
# 환경변수 세팅 (그런데 왜 하는걸까?)
os.environ['GDRIVE_BERT_ROOT'] = GDRIVE_BERT_ROOT
os.environ['MAX_SEQ_LENGTH'] = str(MAX_SEQ_LENGTH)
os.environ['VOCAB_FILE'] = VOCAB_FILE
os.environ['BERT_CONFIG'] = BERT_CONFIG
os.environ['MODEL_DIR'] = MODEL_DIR
os.environ['PROD_MODEL_DIR'] = PROD_MODEL_DIR
os.environ['NUM_LABELS'] = str(NUM_LABELS)

In [26]:
! ls $GDRIVE_BERT_ROOT/data

category.json  dev.tsv	error_values.tsv  test.tsv  train.tsv


In [ ]:
!python /content/Gdrive/bert_files/bert_model/run_classifier.py \
--task_name=cola \
--do_train=True \
--do_predict=False \
--do_eval=True \
--data_dir=$GDRIVE_BERT_ROOT/data \
--vocab_file=$VOCAB_FILE \
--bert_config_file=$BERT_CONFIG \
--output_dir=$MODEL_DIR \
--max_seq_length=$MAX_SEQ_LENGTH \
--train_batch_size=$BATCH_SIZE \
--learning_rate=2e-5 \
--num_train_epochs=3 \
--num_labels=$NUM_LABELS \
--use_tpu=True \
--tpu_name=$TPU_ADDRESS \
--do_lower_case=False

# 저장된 모델을 Export(Save)

- 학습에 사용한 cateogory_list를 GCP Bucket에 저장

In [33]:
!gsutil cp $GDRIVE_BERT_ROOT/data/category.json $MODEL_DIR/

Copying file://Gdrive/bert_files/data/category.json [Content-Type=application/json]...
/ [1 files][  5.8 KiB/  5.8 KiB]                                                
Operation completed over 1 objects/5.8 KiB.                                      


- save_model.pb 파일은 model_exporter.py에서 생성 (into output폴더)

In [51]:
os.environ['PRE_TRAINED_MODEL_PATH'] = PRE_TRAINED_MODEL_PATH

!python $GDRIVE_BERT_ROOT/model_exporter.py \
--task_name="classify" \
--data_path=$MODEL_DIR \
--bert_path=$PRE_TRAINED_MODEL_PATH \
--labels_num=$NUM_LABELS \
--max_seq_length=$MAX_SEQ_LENGTH \
--export_path=$MODEL_DIR/output



2022-03-10 03:01:05.404750: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-03-10 03:01:05.404992: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b72b6cb2c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-03-10 03:01:05.405029: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-03-10 03:01:05.407620: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-03-10 03:01:05.421849: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-10 03:01:05.421912: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (8cfcbb12266d): /proc/driver/nvidia/version does not exist
going to restore checkpoint





The TensorFlow contrib

# 데모 실행 (in Colab)
- 전 단계에서 저장한 Saved Model 로드

In [52]:
!ls $GDRIVE_BERT_ROOT

bert_model	   modeling.py	    run_classifier.py
data		   optimization.py  run_squad.py
model_exporter.py  __pycache__	    tokenization.py


In [60]:
from tensorflow.saved_model import tag_constants
import sys
sys.path.append(GDRIVE_BERT_ROOT+'/')
sys.path.append(GDRIVE_BERT_ROOT+'/bert_model')

from bert_model import tokenization
from bert_model import modeling
from bert_model import run_classifier
import base64
import json

def load_model(model_path):
    '''
    Load the frozen pb file then build the graph
    Args:
        model_path : path of the frozen pb file
    return:
        sess : the session of this bert model
    '''
    session_config = tf.ConfigProto(allow_soft_placement=True)
    sess = tf.Session(config=session_config)
    tf.compat.v1.saved_model.loader.load(
        sess, [tf.saved_model.tag_constants.SERVING], model_path)
    print('Model loaded')
    return sess

tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=False)
processor=run_classifier.ColaProcessor()

with open(GDRIVE_BERT_ROOT + '/data/category.json') as cate_f :
    category_list = json.load(cate_f)

label_list = [str(x) for x in range(len(category_list))]

vocab=tokenization.load_vocab(VOCAB_FILE)
print('Vocab Size :', len(vocab))

session = load_model(MODEL_DIR + '/output')

Vocab Size : 32000
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://nlp_pretrain_model/bert_model/new_model/output/variables/variables
Model loaded


# 샘플 쿼리로 적용

In [ ]:
import numpy as np

def bert_prediction(sess, s):
    predict_example, label_list_test = processor.get_test_examples_n_labels(s)
    frozen_input_ids = sess.graph.get_tensor_by_name('input_ids:0')
    frozen_input_mask = sess.graph.get_tensor_by_name('input_mask:0')
    frozen_segment_ids = sess.graph.get_tensor_by_name('segment_ids:0')
    predict_info = sess.graph.get_tensor_by_name('loss/Softmax:0')

    for (ex_index, example) in enumerate(predict_example):
        if ex_index % 10000 == 0:
            tf.logging.info('Writing example %d of %d' %(ex_index, len(predict_example)))

        feature = run_classifier.convert_single_example(ex_index, example, label_list_test, MAX_SEQ_LENGTH, tokenizer)
        probabilities = sess.run(predict_info,
                                feed_dict={
                                    frozen_input_ids: np.expand_dims(feature.input_ids, 0),
                                    frozen_input_mask: np.expand_dims(feature.input_mask, 0),
                                    frozen_segment_ids: np.expand_dims(feature.segment_ids, 0),
                                })

        # lebel 얻기
        label_pob = []
        for i in probabilities[0]:
            label_pob.append(float(i))

        # 내림차순 정렬
        label_sorted = sorted(label_pob, reverse=True)

        # 1순위, 2순위  반환 값(확률) 확인하기
        first_index, second_index = label_pob.index(label_sorted[0]), label_pob.index(label_sorted[1])

        # label 반환
        first_label, second_label = label_list[first_index], label_list[second_index]

        print('#### 원문 질의 :', s)
        print(f'#### 자동분류 1순위 : {first_label} ({label_sorted[0]})')
        print(f'#### 자동분류 2순위 : {second_label} ({label_sorted[1]})')

        return first_label, second_label

# 샘플 쿼리
sentence = ["저희 집에는 8개월된 푸들한마리가 제 오빠의 옷을 물다가이빨이 빠졋어요.저절로 빠진거에요 옷을 물어서 빠진거에요?", 
             "걍아지가 털이 자꾸 빠져요","강쥐가 중심을 못 잡고 넘어져요", 
             "고양이가 밥을 안 먹어요.어떻게 하죠?",
             "강아지가 켁켁거리면서 밥을 안먹어요"]

for s in sentence :
    result1, result2 = bert_prediction(session, s)
    print(f'## 자동분류명 1순위 : {category_list[int(result1)]}, ## 자동분류명 2순위 : {category_list[int(result2)]}')

